In [2]:
import requests
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [1]:
ACCESS_TOKEN = 'ACCESS-TOKEN'

In [3]:
def search_song_on_spotify(song_name):
    headers = {
        'Authorization': f'Bearer {ACCESS_TOKEN}'
    }
    url = f'https://api.spotify.com/v1/search?q={song_name}&type=track&limit=50'

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        tracks = data['tracks']['items']
        return [(track['id'], track['name'], track['artists'][0]['name']) for track in tracks]
    else:
        print(f"Error fetching data: {response.status_code}")
        return []

In [4]:
def get_audio_features(track_ids):
    headers = {
        'Authorization': f'Bearer {ACCESS_TOKEN}'
    }
    ids = ','.join(track_ids)
    url = f'https://api.spotify.com/v1/audio-features?ids={ids}'

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data['audio_features']
    else:
        print(f"Error fetching audio features: {response.status_code}")
        return []

In [11]:
def analyze_bass_characteristics(track_name):
    results = search_song_on_spotify(track_name)

    if not results:
        print("No tracks found.")
        return

    # Remove exact duplicates based on track name
    unique_results = []
    seen_tracks = set()
    for track in results:
        track_name = track[1].lower()
        if track_name not in seen_tracks:
            unique_results.append(track)
            seen_tracks.add(track_name)

    # Proceed with unique results
    track_ids = [track[0] for track in unique_results]

    audio_features = get_audio_features(track_ids)

    audio_features = [features for features in audio_features if features is not None]

    if not audio_features:
        print("No audio features found.")
        return

    # Extract all features (except liveness)
    feature_vectors = np.array([[features['danceability'],
                                 features['energy'],
                                 features['valence'],
                                 features['loudness'],
                                 features['key'],
                                 features['mode'],
                                 features['speechiness'],
                                 features['acousticness'],
                                 features['instrumentalness'],
                                 features['tempo'],
                                 features['time_signature']]
                                for features in audio_features])

    # Prioritize energy, danceability, and valence
    prioritized_features = feature_vectors.copy()
    prioritized_features[:, 0] *= 2  # Danceability
    prioritized_features[:, 1] *= 2  # Energy
    prioritized_features[:, 2] *= 2  # Valence

    # Calculate similarities
    similarities = cosine_similarity(prioritized_features)

    # Exclude the query track's similarity to itself
    similarities = similarities[0][1:]  # Exclude self similarity

    # Find the most similar tracks (excluding itself)
    most_similar_indices = np.argsort(similarities)[-10:][::-1]

    slowed_sped_tracks = []
    different_songs_tracks = []

    # Categorize tracks based on their names
    for index in most_similar_indices:
        similar_track = unique_results[index]
        track_name, artist_name = similar_track[1], similar_track[2]

        # Check if the track name contains keywords indicating slowed or sped versions
        if any(keyword in track_name.lower() for keyword in ['slowed', 'sped up', 'reverbed']):
            slowed_sped_tracks.append(f"{track_name} by {artist_name}")
        else:
            different_songs_tracks.append(f"{track_name} by {artist_name}")

    print(f"Top Similar Tracks to '{track_name}':")

    if slowed_sped_tracks:
        print("\nSlowed/Sped/Reverbed Versions:")
        for track in slowed_sped_tracks:
            print(f"- {track}")

    if different_songs_tracks:
        print("\nDifferent Similar Songs:")
        for track in different_songs_tracks:
            print(f"- {track}")

In [12]:
track_name_input = input("Enter a song name to analyze its bass characteristics on Spotify: ")
analyze_bass_characteristics(track_name_input)

Enter a song name to analyze its bass characteristics on Spotify: Feel It Still
Top Similar Tracks to 'Feel It (Invincible)':

Different Similar Songs:
- It's Too Late by Carole King
- Feel It Still - Gryffin Remix by Portugal. The Man
- feel it all by sød ven
- Feel It Still - ZHU Remix by Portugal. The Man
- All Nightmare Long by Metallica
- Feel It Still - Medasin Remix by Portugal. The Man
- Feel it still (Inkie remake) by INKIE
- Feel It Still - Ofenbach Remix by Portugal. The Man
- Kids in Love (feat. The Night Game) by Kygo
- Feel It (Invincible) by ONE PROJECT
